In [ ]:
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/c1666ca7a183eece7073c4c7e0946b8f46bfe1ed/bbc-fulltext.zip
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/796491820603626de2fcfec6ae69fef3937a1866/bbc.zip
!wget /content/drive/MyDrive/Colab Notebooks/bbc.zip

/content/drive/MyDrive/Colab: Scheme missing.
--2022-12-02 09:22:25--  http://notebooks/bbc.zip
Resolving notebooks (notebooks)... failed: Name or service not known.
wget: unable to resolve host address ‘notebooks’


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [84]:
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
wandb: Currently logged in as: vemul-vamsy (grp_11). Use `wandb login --relogin` to force relogin


In [91]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from urllib import request
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import glob
import string

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
!pip3 install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix , classification_report

In [94]:
import wandb
from wandb.keras import WandbMetricsLogger
from wandb.keras import WandbCallback
wandb.init(project="2A_BBC_BERT", entity="grp_11",job_type="train")

In [95]:
bbc_path = "/content/drive/MyDrive/Colab_Notebooks/bbc/"

In [96]:
def readFiles(path,clsId):
  fileNames = []
  fNames = []
  articles = []
  clsIds = []
  #print(glob.glob(path+"*.txt"))
  filenames=os.listdir(path)
  
  for ef in filenames:
    f = open(path+ef).readlines()
    articles.append(" ".join(f))
    clsIds.append(clsId)
    fNames.append(ef)
  
  return fNames,articles,clsIds
  

In [97]:
classNames = ["business","entertainment","politics","sport","tech"]
allArticles = []
allClsIds = []
allFileNames = []
for idx in range(len(classNames)):
  p = bbc_path+classNames[idx]+"/"
  fn,ad,cid = readFiles(p,idx)
  allArticles = allArticles + ad
  allClsIds = allClsIds + cid
  allFileNames = allFileNames + fn
 
print(len(allArticles),len(allClsIds),len(allFileNames))

2225 2225 2225


In [98]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [99]:
for i in range(len(allArticles)):
  allArticles[i] = allArticles[i].lower() #convert to lower case
  allArticles[i] = re.sub('<.*?>',' ',allArticles[i]) # remove html tags
  allArticles[i] = re.sub('[^a-zA-Z]',' ',allArticles[i]) #remove non-alphabet
  allArticles[i] = re.sub(r'\s+[a-zA-Z]\s+',' ',allArticles[i]) #remove single character
  allArticles[i] = re.sub(r'\s+', ' ',allArticles[i]) #remvoe multiple spaces
  allArticles[i] = REPLACE_BY_SPACE_RE.sub(' ',allArticles[i]) # replace REPLACE_BY_SPACE_RE symbols by space in text
  allArticles[i] = BAD_SYMBOLS_RE.sub('',allArticles[i]) #remove symbols that are in BAD_SYMBOLS from text
  allArticles[i] = ' '.join(word for word in allArticles[i].split() if word not in STOPWORDS) # remove stop words

In [100]:
allArticles[0]

'us interest rate rise expected us interest rates expected rise fifth time since june following us federal reserve latest rate setting meeting later tuesday borrowing costs tipped rise quarter percentage point move comes recovery us economy world biggest shows signs robustness sustainability dollar record breaking decline meanwhile spooked markets along high oil prices raised concerns pace inflation seeing evidence inflation moving higher said ken kim analyst stone mccarthy research risk actually happening mr kim added borrowing costs could rise fed said move measured way combat price growth lift interest rates year lows prompted sluggish us global growth economic picture looking rosy fed implemented quarter percentage point rises june august september november although us economy grew annual rate three months september analysts warn fed careful move aggressively take wind recovery sails earlier month figures showed job creation still weak consumer confidence subdued think fed feels fa

In [101]:
#df with word2vec
cols = ['article', 'labels']
d = {'article':allArticles, 'labels':allClsIds}
df = pd.DataFrame(data=d, columns=cols)
df.head()

,article,labels
0,us interest rate rise expected us interest rat...,0
1,uk economy facing major risks uk manufacturing...,0
2,ask jeeves tips online ad revival ask jeeves b...,0
3,us company admits benin bribery us defence tel...,0
4,ticking budget facing us budget proposals laid...,0


In [102]:
val_counts = df['labels'].value_counts()
val_counts

3    511
0    510
2    417
4    401
1    386
Name: labels, dtype: int64

In [103]:
df_0 = df[df['labels']==0]
df_1 = df[df['labels']==1]
df_2 = df[df['labels']==2]
df_3 = df[df['labels']==3]
df_4 = df[df['labels']==4]

print(df_0.shape)
print(df_1.shape)
print(df_2.shape)
print(df_3.shape)
print(df_4.shape)

(510, 2)
(386, 2)
(417, 2)
(511, 2)
(401, 2)


In [104]:
min_size_val = df_1.shape[0]
df_0_downsampled = df_0.sample(min_size_val)
df_2_downsampled = df_2.sample(min_size_val)
df_3_downsampled = df_3.sample(min_size_val)
df_4_downsampled = df_4.sample(min_size_val)

print(df_0_downsampled.shape)
print(df_2_downsampled.shape)
print(df_3_downsampled.shape)
print(df_4_downsampled.shape)

(386, 2)
(386, 2)
(386, 2)
(386, 2)


In [105]:
df_balanced = pd.concat([df_0_downsampled,df_1,df_2_downsampled,df_3_downsampled,df_4_downsampled])
df_balanced.shape

(1930, 2)

In [58]:
df_balanced.head()

,article,labels
142,glaxo aims high profit fall glaxosmithkline sa...,0
316,indy buys india paper irish publishing group i...,0
262,card fraudsters targeting web new safeguards c...,0
107,bt offers equal access rivals bt moved pre emp...,0
304,sec rethink post enron rules us stock market w...,0


In [106]:
df_balanced['labels'].value_counts()

0    386
1    386
2    386
3    386
4    386
Name: labels, dtype: int64

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['article'],df_balanced['labels'],stratify=df_balanced['labels'])
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1447,) (483,) (1447,) (483,)


In [108]:
print("Train split by classes\n",y_train.value_counts(),"\n\n", "Test split by classes\n",y_test.value_counts())

Train split by classes
 1    290
3    290
2    289
0    289
4    289
Name: labels, dtype: int64 

 Test split by classes
 0    97
4    97
2    97
3    96
1    96
Name: labels, dtype: int64


## BERT - Bidirectional Encoder Representations from Transformers

In [109]:

# Downloading preprocessing files and model from the TF models repository. This will take more time for dowloading the files
preprocessing_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/2')

In [111]:
def build_classifier_model():
  text_input = keras.layers.Input(shape=(), dtype=tf.string, name='text')
  
  #pre-processing the text input
  encoded_inputs = preprocessing_layer(text_input)

  #pass encoded_inputs through the BERT model
  outputs = bert_encoder(encoded_inputs);
  
  #Model returns embedding of the entire dataset
  net = outputs['pooled_output']
  
  #pass the entire training data embeddings to the dropout layer
  net = keras.layers.Dropout(0.2, name="dropout")(net)

  #dense layer
  net = keras.layers.Dense(5, activation='softmax', name='output')(net)
  results = keras.Model(text_input, net)
  return results


In [112]:
model = build_classifier_model()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [113]:
wandb_callback = WandbCallback(monitor="val_loss", log_weights=True, log_evaluation=True,validation_steps=5)

In [114]:
#compile the model
model.compile(optimizer ='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])



In [115]:
history = model.fit(X_train, 
                    y_train, 
                    epochs = 10,
                    callbacks=[wandb_callback,WandbMetricsLogger()], 
                    validation_data=(X_test,y_test))

wandb: WARNING Error initializing ValidationDataLogger in WandbCallback. Skipping logging validation data. Error: 


Epoch 1/10
45/46 [============================>.] - ETA: 0s - loss: 1.3478 - accuracy: 0.4694

wandb: Adding directory to artifact (/content/wandb/run-20221216_174226-cjkuag0j/files/model-best)... Done. 1.3s


46/46 [==============================] - 24s 445ms/step - loss: 1.3443 - accuracy: 0.4706 - val_loss: 0.8691 - val_accuracy: 0.8033
Epoch 2/10
45/46 [============================>.] - ETA: 0s - loss: 0.7191 - accuracy: 0.8076

wandb: Adding directory to artifact (/content/wandb/run-20221216_174226-cjkuag0j/files/model-best)... Done. 1.3s


...

46/46 [==============================] - 20s 431ms/step - loss: 0.2422 - accuracy: 0.9371 - val_loss: 0.2298 - val_accuracy: 0.9482
Epoch 9/10
45/46 [============================>.] - ETA: 0s - loss: 0.2271 - accuracy: 0.9403

wandb: Adding directory to artifact (/content/wandb/run-20221216_174226-cjkuag0j/files/model-best)... Done. 1.3s


46/46 [==============================] - 20s 430ms/step - loss: 0.2268 - accuracy: 0.9406 - val_loss: 0.2170 - val_accuracy: 0.9462
Epoch 10/10
45/46 [============================>.] - ETA: 0s - loss: 0.2087 - accuracy: 0.9431

wandb: Adding directory to artifact (/content/wandb/run-20221216_174226-cjkuag0j/files/model-best)... Done. 1.3s


46/46 [==============================] - 19s 421ms/step - loss: 0.2095 - accuracy: 0.9426 - val_loss: 0.2094 - val_accuracy: 0.9462


In [116]:
wandb.finish()

accuracy,▁▆▇▇██████
epoch,▁▂▃▃▄▅▆▆▇█
epoch/accuracy,▁▆▇▇██████
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▂▁▁▁▁
epoch/val_accuracy,▁▆▆█▇█████
epoch/val_loss,█▅▃▂▂▂▁▁▁▁
loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▆▆█▇█████
val_loss,█▅▃▂▂▂▁▁▁▁


In [74]:
model.evaluate(X_test,y_test)

16/16 [==============================] - 3s 142ms/step - loss: 0.2283 - accuracy: 0.9255


[0.22831785678863525, 0.9254658222198486]

In [ ]:
#download trained model to local machine
model.save("/content/drive/MyDrive/Colab Notebooks/2A_BBC_BERT_Trained_Model_2/")

In [ ]:
model_pt = tf.keras.models.load_model(
    "/content/drive/MyDrive/Colab Notebooks/2A_BBC_BERT_Trained_Model_2/"
)

In [ ]:
model_pt.evaluate(X_test,y_test)

16/16 [==============================] - 3s 138ms/step - loss: 0.1752 - accuracy: 0.9482


[0.17523697018623352, 0.9482401609420776]